# mimic LOB - DEMO


# 0. Imports

In [ ]:
# Imports
from   IPython.display import display, HTML
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

# Display options
pd.set_option('display.width', 1000)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('mode.chained_assignment', None)

In [ ]:
import mimicLOB as lob
from mimicLOB.orderbook.orderbook import OrderBook
from mimicLOB.agent.genericAgent import genericAgent

# 1. LOB creation

In [ ]:
# b_tape = True means the LOB 
LOB = OrderBook(tick_size = 1, 
                b_tape    = True,
                verbose   = False)

# 2. Agents
The agents is constantly connected to the distant LOB via the server address.

#### Parameters :
1. distant : if true, the agent must have the server address. If False, he should have the orderbook object passed in the configuration.
2. server : when distant is True, the http address of the server the LOB is running on.
3. id : optional, default is generic. the if is used in the transaction tape.
4. b_record : optional, boolean, default is False. If True, the agent records all his activity : sent orders, executed trades, pending orders ...


In [ ]:
EURONEXT_config = {'orderbook'  : LOB,
                   'id'         : 'market'}
                 
myself_config = {'orderbook'  : LOB,
                 'id'         : 'FDR',
                 'b_record'   : True}

otherguy_config = {'orderbook'  : LOB,
                   'id'         : 'Olivier'}

euronext = genericAgent(**EURONEXT_config)
myself   = genericAgent(**myself_config)
other    = genericAgent(**otherguy_config)

In [ ]:
# Only agents with b_record = True
LOB.agentList

#### The agent with id 'market' can change the LOB configuration and reset it

In [ ]:
euronext.setLOB_tickSize(1)

# 3. Get the LOB state
Either agents have access to LOB information

In [ ]:
LOBstate = myself.getLOBState()
tickSize =  float(myself.getTickSize())
print(f'tick size : {tickSize}')
if type(LOBstate) == pd.DataFrame:
    if len(LOBstate) >0 :
        LOBstate = LOBstate.set_index('Price').sort_index()
        minPrice = LOBstate.index[0]
        maxPrice = LOBstate.index[-1]
        LOBstate = LOBstate.reindex(np.arange(minPrice, maxPrice+tickSize, tickSize))
        LOBstate.plot.bar(figsize=(20, 7))
    else:
        print('LOB is empty')
else:
    display(LOBstate)

# 4. Send orders

#### limit sell order @101, 102 & limit buy order @99, 100
Check the LOB state afterwards

In [ ]:
# Send Orders
myself.send_sell_limit_order(quantity = 100,
                             price    = 101)

myself.send_sell_limit_order(quantity = 200,
                             price    = 102)

other.send_buy_limit_order(quantity   = 100,
                             price    = 100)

other.send_buy_limit_order(quantity   = 100,
                             price    = 99)
print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

#### market orders

We send a 150 sell market order. 100 will be executed @100 & 50 @99 

In [ ]:
myself.send_sell_market_order(quantity   = 150)

print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

#### match orders
Olivier will buy 150 at 101. A transaction will happen, and the new best bid will be 101. Check LOB State.

The agent 'FDR' is notified of the transaction. Pending orders & transaction list are updated.

In [ ]:
other.send_buy_limit_order(quantity = 150,
                           price    = 101)

print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

# 5. Modify orders

Rules for modification : 
1. If the order is already executed : no new order
2. Any order entered into the Central Order Book may be modified or cancelled prior to its execution. Any increase in the order quantity or change in the limit price shall cause the forfeiture of time priority. (Rule 4202/4 of Euronext)

Rules for cancellation :
1. If the order is already executed : no order cancellation

#### modify an order

we modify an order to make it aggressive (from 102 to best bid = 101).
It will result in a traed of 50 shares + 150 shares will sit in the LOB at the best ask.

In [ ]:
print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

In [ ]:
myself.modifyOrder(order_id     = 2, 
                   side         = 'ask',
                   new_price    = 101,
                   new_quantity = 200)

print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

#### Cancel an order

In [ ]:
print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

In [ ]:
myself.cancelOrder(side     = 'ask', 
                   order_id = 2)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

# 6. Modify The LOB
One agent type (market) can reset the LOB, dump the tape, and modify the LOB properties 
#### Change tick size

In [ ]:
euronext.setLOB_tickSize(0.1)

#### Remove all pending orders

In [ ]:
euronext.resetLOB_PendingOrders()

#### Remove orders and transactions

In [ ]:
euronext.resetLOB()